In [ ]:
from sys import path
base_path = "/media/paulati/Nuevo vol/paula/ingebi/2020/agustina_mazzella/github/arabidopsis_phospho"
path.append(base_path)    

from preparation import util

In [ ]:
from os import path, remove
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics as stats
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve, f1_score, confusion_matrix, recall_score, precision_score

In [ ]:
musite_results_base_path = path.join(base_path, 'data/results/musitedeep')
musiteDeep_prediction_score_by_id_base_file_path = path.join(musite_results_base_path, "Prediction_results_score_by_id_base.txt")

In [ ]:
musite_scores_file_path = musiteDeep_prediction_score_by_id_base_file_path.replace(".txt", ".zip")
musite_scores_data = pd.read_csv(musite_scores_file_path, sep = "\t")
musite_scores_data

In [ ]:
experimental_file_path = path.join(base_path, "data/preproc/experimental_ids.zip")
experimental_data = pd.read_csv(experimental_file_path, sep = "\t")

In [ ]:
musite_scores_data.index = musite_scores_data.protein_id_base
experimental_data.index = experimental_data.code
musite_experimental_data = musite_scores_data.join(experimental_data, how = "outer")
#musite_experimental_data

In [ ]:
musite_experimental_data["label"] = [1 if x is not np.NaN else 0 for x in musite_experimental_data.code.values]
musite_experimental_data["color"] = ["orange" if x == 1 else "blue" for x in musite_experimental_data.label.values]

In [ ]:
musite_experimental_data

In [ ]:
import seaborn as sns

In [ ]:
musite_experimental_data_sort = musite_experimental_data.sort_values(by="min_score", ascending= True)
sns.scatterplot(x = range(musite_experimental_data_sort.shape[0]), y = musite_experimental_data_sort.min_score, 
                hue = musite_experimental_data_sort.label.values, style = musite_experimental_data_sort.label.values)

In [ ]:
musite_experimental_data_sort = musite_experimental_data.sort_values(by="max_score", ascending= True)
sns.scatterplot(x = range(musite_experimental_data_sort.shape[0]), y = musite_experimental_data_sort.max_score,
                hue = musite_experimental_data_sort.label.values, style = musite_experimental_data_sort.label.values)

In [ ]:
musite_experimental_data_sort = musite_experimental_data.sort_values(by="mean_score", ascending= True)
sns.scatterplot(x = range(musite_experimental_data_sort.shape[0]), y = musite_experimental_data_sort.mean_score,
                hue = musite_experimental_data_sort.label.values, style = musite_experimental_data_sort.label.values)

In [ ]:
musite_experimental_data.label.value_counts()

In [ ]:
y = musite_experimental_data.label

probs = musite_experimental_data.max_score

fpr, tpr, thresholds = roc_curve(y, probs)

auc = roc_auc_score(y, probs)

In [ ]:
auc

## ROC Curve

In [ ]:
# generate a no skill prediction (majority class)
ns_probs = [0 for _ in range(musite_experimental_data.shape[0])]
# keep probabilities for the positive outcome only
musite_probs = musite_experimental_data.max_score

In [ ]:
# calculate scores
ns_auc = roc_auc_score(musite_experimental_data.label, ns_probs)
musite_auc = roc_auc_score(musite_experimental_data.label, musite_probs)
# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (musite_auc))
# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(musite_experimental_data.label, ns_probs)
musite_fpr, musite_tpr, _ = roc_curve(musite_experimental_data.label, musite_probs)
# plot the roc curve for the model
plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
plt.plot(musite_fpr, musite_tpr, marker='.', label='MusteDeep')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()

## Optimal threshold

1.

This code evaluates `thresholds_count` samples looking for optimal threshold.

`data_frac` specify the proportion of the dataset records evaluated in each sample.

Optimal threshold as the maximum `abs(tpr-fpr)`

In [ ]:
def calculate_j_score(tpr, fpr, score_type):
    if score_type == 1:
        j_scores = abs(tpr-fpr)
    elif score_type == 2:
        #j_scores = np.sqrt(((1-tpr) * (1-tpr)) + (fpr * fpr))
        j_scores = np.sqrt(((1-tpr) * (1-tpr)) + (fpr * fpr))
        j_scores = -j_scores
    return j_scores

    

def calculate_optimal_thresholds(pred_exp_data, thresholds_count, data_frac, score_type=1):
    
    result = []
    
    exp_pos_mask = pred_exp_data.label == 1
    exp_pos = pred_exp_data.loc[exp_pos_mask, :]
    prop_exp_pos = exp_pos_mask.sum() / pred_exp_data.shape[0]
    
    exp_neg_mask = pred_exp_data.label == 0
    exp_neg = pred_exp_data.loc[exp_neg_mask, :]
    prop_exp_neg = exp_neg_mask.sum() / pred_exp_data.shape[0]
    
    pos_sample_count = round(data_frac * prop_exp_pos * pred_exp_data.shape[0])
    print(pos_sample_count)
    
    neg_sample_count = round(data_frac * prop_exp_neg * pred_exp_data.shape[0])
    print(neg_sample_count)
    
    for i in range(thresholds_count):
    
        sample_pos = exp_pos.sample(pos_sample_count, replace = True, axis = 0 )
        
        #print(sample_pos.shape[0])
        
        sample_neg = exp_neg.sample(neg_sample_count, replace = True, axis = 0 )
        
        #print(sample_neg.shape[0])
    
        sample = pd.concat([sample_pos, sample_neg], axis = 0)
        
        #print(sample.shape[0])
    
        y = pd.Categorical(sample.label.values)
    
        scores = sample.max_score.values
    
        fpr, tpr, thresholds = roc_curve(y, scores, pos_label=1)
    #roc_auc = metrics.auc(fpr, tpr)
    
        #j_scores = abs(tpr-fpr)
        j_scores = calculate_j_score(tpr, fpr, score_type)        
    
        optimal_idx = np.argmax(j_scores)
    
        optimal_threshold = thresholds[optimal_idx]
        
        result.append(optimal_threshold)
    
    
    return(result)
    

In [ ]:
optimal_thresholds = calculate_optimal_thresholds(musite_experimental_data, 5000, 0.3, 1)
print(np.mean(optimal_thresholds))
print(np.std(optimal_thresholds))

In [ ]:
sns.histplot(optimal_thresholds)

In [ ]:
optimal_1 = stats.mode(optimal_thresholds)
optimal_1

Without sampling:

In [ ]:
optimal_thresholds = calculate_optimal_thresholds(musite_experimental_data, 1, 1, 1)
optimal_thresholds

2.

Pick the threshold t∗ that minimizes the distance to the top left can be done by measuring its distance 

$distance = (0, 1) - (fpr, tpr)$

$= (-fpr, 1-tpr)$

$= \sqrt{(-fpr)^2 + (1-tpr)^2}$


$t∗ = argmin_t(\sqrt{fpr_t^2 + (1 - tpr_t)^2}$

$t∗ = argmax_t(- \sqrt{fpr_t^2 + (1 - tpr_t)^2}$

In [ ]:
optimal_thresholds = calculate_optimal_thresholds(musite_experimental_data, 5000, 0.3, 2)
print(np.mean(optimal_thresholds))
print(np.std(optimal_thresholds))

In [ ]:
sns.histplot(optimal_thresholds)

In [ ]:
optimal_2 = stats.mode(optimal_thresholds)
optimal_2

In [ ]:
optimal_thresholds = calculate_optimal_thresholds(musite_experimental_data, 1, 1, 2)
optimal_thresholds

## Confusion matrix

In [ ]:
y_true = musite_experimental_data.label

In [ ]:
optimal_threshold_1 = optimal_1
y_pred = [1 if x >= optimal_threshold_1 else 0 for x in musite_experimental_data.max_score.values]
confusion_matrix(y_true, y_pred)

In [ ]:
optimal_threshold_2 = optimal_2
y_pred = [1 if x >= optimal_threshold_2 else 0 for x in musite_experimental_data.max_score.values]
confusion_matrix(y_true, y_pred)

In [ ]:
default_threshold = 0.5
y_pred = [1 if x >= default_threshold else 0 for x in musite_experimental_data.max_score.values]
confusion_matrix(y_true, y_pred)

**Threshold is set in 0.852**

## Precision-Recall

<img src='./img/800px-Precisionrecall.svg.png' width= 20%></img>

source: https://en.wikipedia.org/wiki/Precision_and_recall#/media/File:Precisionrecall.svg

The precision-recall curve shows the tradeoff between precision and recall for different threshold. 

A high area under the curve represents both high recall and high precision, where high precision relates to a low false positive rate, and high recall relates to a low false negative rate. 

High scores for both show that the classifier is returning accurate results (high precision), as well as returning a majority of all positive results (high recall).

source: https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html

In [ ]:
musite_precision, musite_recall, _ = precision_recall_curve(musite_experimental_data.label, musite_probs)
# calculate scores

y_pred = [1 if x >= optimal_1 else 0 for x in musite_experimental_data.max_score.values]

musite_f1 = f1_score(musite_experimental_data.label, y_pred)

musite_auc = auc(musite_recall, musite_precision)
# summarize scores
print('MusiteDeep: f1=%.3f auc=%.3f' % (musite_f1, musite_auc))
# plot the precision-recall curves
no_skill = len(musite_experimental_data.label == 1) / len(musite_experimental_data.label)
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='Null Model')
plt.plot(musite_recall, musite_precision, marker='.', label='MusiteDeep')
# axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')
# show the legend
plt.legend()
# show the plot
plt.show()


In [ ]:
threshold_count = 100
thresholds = []
for i in range(threshold_count):
    threshold = i / thr_count
    thresholds.append(threshold)

f1_scores = []
recall_scores = []
precision_scores = []
y_true = musite_experimental_data.label
    
for threshold in thresholds:
    y_pred = [1 if x >= threshold else 0 for x in musite_experimental_data.max_score.values]
    f1 = f1_score(y_true, y_pred)    
    recall = recall_score(y_true, y_pred)    
    precision = precision_score(y_true, y_pred)    
    f1_scores.append(f1)
    recall_scores.append(recall)
    precision_scores.append(precision)

In [ ]:
sns.histplot(precision_scores, color="steelblue")
sns.histplot(recall_scores, color="orange")
sns.histplot(f1_scores, color = "darkred")

In [ ]:
sns.scatterplot(x = thresholds, y = f1_scores)

In [ ]:
max(f1_scores)

In [ ]:
index = np.argmax(f1_scores)
print(recall_scores[index])
print(precision_scores[index])

Metrics with optimal_1 threshold :

In [ ]:
y_pred = [1 if x >= optimal_1 else 0 for x in musite_experimental_data.max_score.values]

f1 = f1_score(y_true, y_pred)    
recall = recall_score(y_true, y_pred)    
precision = precision_score(y_true, y_pred)    

print(f1)
print(recall)
print(precision)



## References

---

https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/

https://stats.stackexchange.com/questions/123124/how-to-determine-the-optimal-threshold-for-a-classifier-and-generate-roc-curve

http://www0.cs.ucl.ac.uk/staff/W.Langdon/roc/


In [ ]:
optimal_1